[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pdf-tools/components-code-sample-hub/blob/main/jupyter/pdftools_toolbox/pdftools_toolbox_add_vector_graphics.ipynb)

In [ ]:
%pip install https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/productkits/PDFSDKXT/latest/pdftools_toolbox-latest.tar.gz
%pip install ipython

# Add vector graphic to PDF
Draw a line on an existing PDF page.

In [ ]:
import io
from pdftools_toolbox.pdf import Document, FileReference, Metadata, Page, PageCopyOptions
from pdftools_toolbox.geometry.real import Point
from pdftools_toolbox.pdf.content import ColorSpace, ContentGenerator, IccBasedColorSpace, Paint, ProcessColorSpaceType, Stroke, Path, PathGenerator
from pdftools_toolbox.pdf.navigation import ViewerSettings

In [ ]:
# Download a file from a given URL and save it to the local system
def prepare_file(url: str, path: str):
    import requests
    response = requests.get(url)
    response.raise_for_status()

    with open(path, 'wb') as f:
        f.write(response.content)

In [ ]:
# Set input arguments
input_url = 'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/GraphicsNone.pdf'
input_file_path = 'GraphicsNone.pdf'
prepare_file(input_url, input_file_path)
output_file_path = 'GraphicsWithLine.pdf'

In [ ]:
def copy_document_data(in_doc: Document, out_doc: Document):
    # Copy document-wide data

    # Output intent
    if in_doc.output_intent is not None:
        in_doc.output_intent = IccBasedColorSpace.copy(out_doc, in_doc.output_intent)

    # Metadata
    out_doc.metadata = Metadata.copy(out_doc, in_doc.metadata)

    # Viewer settings
    out_doc.viewer_settings = ViewerSettings.copy(out_doc, in_doc.viewer_settings)

    # Associated files (for PDF/A-3 and PDF 2.0 only)
    outAssociatedFiles = out_doc.associated_files
    for in_file_ref in in_doc.associated_files:
        outAssociatedFiles.append(FileReference.copy(out_doc, in_file_ref))

    # Plain embedded files
    out_embedded_files = out_doc.plain_embedded_files
    for in_file_ref in in_doc.plain_embedded_files:
        out_embedded_files.append(FileReference.copy(out_doc, in_file_ref))

In [ ]:
def add_line(out_doc: Document, page: Page):
    # Extract content generator
    with ContentGenerator(page.content, False) as generator:

        # Create a path
        path = Path()
        with PathGenerator(path) as path_generator:
            # Draw a line diagonally across the page
            page_size = page.size
            path_generator.move_to(Point(x = 10.0, y = 10.0))
            path_generator.line_to(Point(x = page_size.width - 10.0, y=page_size.height - 10.0))

        # Create a RGB color space
        device_rgb_color_space = ColorSpace.create_process_color_space(out_doc, ProcessColorSpaceType.RGB)

        # Create a red color
        red = [1.0, 0.0, 0.0]

        #  Create a paint
        paint = Paint.create(out_doc, device_rgb_color_space, red, None)

        # Create stroking parameters with given paint and line width
        stroke = Stroke(paint, 10.0)

        # Draw the path onto the page
        generator.paint_path(path, None, stroke)

In [ ]:
try:
    # Set and check license key. If the license key is not valid, an exception is thrown.
    from pdftools_toolbox.sdk import Sdk
    Sdk.initialize("INSERT-LICENSE-KEY", None)

    # Open input document
    with io.FileIO(input_file_path, 'rb') as in_stream:
        with Document.open(in_stream, None) as in_doc:
    
            # Create output document
            with io.FileIO(output_file_path, 'wb+') as output_stream:
                with Document.create(output_stream, in_doc.conformance, None) as out_doc:
    
                    # Copy document-wide data
                    copy_document_data(in_doc, out_doc)
    
                    # Define page copy options
                    copy_options = PageCopyOptions()
    
                    # Copy all pages from input document
                    for in_page in in_doc.pages:
                        out_page = Page.copy(out_doc, in_page, copy_options)
    
                        # Add a line
                        add_line(out_doc, out_page)
    
                        # Add page to output document
                        out_doc.pages.append(out_page)

    print("Execution successful.")
except Exception as e:
    print(f"An error occurred: {e}")